## Objective 
This script process each of the vessel locations independently to infer the coordinates where these ships have been stationary for a period greater than 2 hours. To infer the stationary locations for each of the vessels the steps are as follows:-

* Order vessel locations by timestamp
* Determine the haversine distance successive known locations.
* Infer the speed based on the distance and time-stamps.
* A ships is deemed stationary if the speed of the vessel is less than $ 1.0 $ km/hr for a time period greater than 2 hours.
* Finally to ensure that the clustering algorithm in the next script runs smoothly for the entire dataset covering the whole globe, the stationary ship locations are partitioned at geohash level 3 and stored as separate files. The clustering algorithm can be applied to each geohash independently.

#### Libraries used 
Pandas, joblib, pygeohash

In [1]:
import pandas as pd
import numpy as np
import datetime
from pyathena import connect
from pyathena.pandas_cursor import PandasCursor
from joblib import Parallel, delayed
import folium
from tqdm import tqdm
from math import radians, cos, sin, asin, sqrt
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
import pygeohash as gh

In [2]:
cursor = connect(profile_name="abhinav.sunderrajan",
                     s3_staging_dir='s3://data-science-athena-challenge/athena-output/',
                     schema_name='prophesea_staging',
                     region_name='eu-west-2',
                     work_group="data-candidate-1",
                     cursor_class=PandasCursor).cursor()

In [3]:
query = "select imo from vessel_temp where dwt>150000"
imos = cursor.execute(query).as_pandas()
cursor.close()
imos_list = list(imos.iloc[:, 0])
len(imos_list)

1648

In [4]:
R = 6371.0088

def haversine(lat1, lon1, lat2, lon2):
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R * c

In [5]:
def process_vessel_locations(imo_no):
    ais = pd.read_parquet(f"data/vessel_locations/lrimoshipno={imo_no}")
    ais.sort_values(by="movementdatetime", inplace=True)
    ais.reset_index(drop=True, inplace=True)
    ais["movementdatetime"] = pd.to_datetime(ais["movementdatetime"])
    ais["lrimoshipno"] = imo_no
    ais["date"] = ais.movementdatetime.apply(
        lambda x: f"{x.year}-{x.month:02d}-{x.day:02d}")
    ais["prev_time"] = ais.movementdatetime.shift(1)
    ais["prev_latitude"] = ais.latitude.shift(1)
    ais["prev_longitude"] = ais.longitude.shift(1)
    ais["time_interval"] = (ais["movementdatetime"] -
                            ais["prev_time"]).apply(lambda x: x.seconds)
    ais["distance_km"] = ais.apply(lambda row: haversine(
        row["latitude"], row["longitude"], row["prev_latitude"], row["prev_longitude"]), axis=1)
    ais["speed_km_hr"] = ais["distance_km"]*3600/ais["time_interval"]
    
    # stationary for at least two hours
    ports = ais[(ais.speed_km_hr < 1) & (ais.time_interval> (3600*2))].copy()
    ports['geohash'] = ports.apply(lambda x: gh.encode(
    x.latitude, x.longitude, precision=3), axis=1)

    ports.reset_index(drop=True, inplace=True)
    if ports.shape[0] > 0:
        ports[["latitude", "longitude", "date", "lrimoshipno","geohash"]].to_parquet(
            f"data/port_locations", partition_cols=["geohash"], engine='pyarrow', compression='gzip')

In [6]:
Parallel(n_jobs=4)(delayed(process_vessel_locations)(ship_no)
                   for ship_no in tqdm(imos_list))

100%|██████████| 1648/1648 [19:06<00:00,  1.44it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,